Import essential libraries

In [83]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.nn as nn

Assume vector embeddings if length "3" for words in sentence "Your journey starts with one step" for simplicity

In [84]:
inputs = torch.tensor([
 [0.43, 0.15, 0.89],
 [0.55, 0.87, 0.66],
 [0.57, 0.85, 0.64],
 [0.22, 0.58, 0.33],
 [0.77, 0.25, 0.10],
 [0.05, 0.80, 0.55]
]
)

Considering only second token for calculation of attention weights

x_2 is the second element

d_in is input embedding size

d_out is output embedding size

In [85]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

Initialize the weight matrices

In [86]:
torch.manual_seed(0)
W_query = torch.nn.Parameter(torch.randn(d_in, d_out),requires_grad=False)
W_key = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

Compute key,query and value

Obtain all keys and values with Matrix Multiplication

In [87]:
queries = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value
print("Query:", queries.shape)

Query: torch.Size([6, 2])


Compute attention scores

In [88]:
atten_scores = queries @ keys.T
print("Attention Scores:", atten_scores.shape)

Attention Scores: torch.Size([6, 6])


Scale attention wieghts and apply softmax

In [89]:
d_k = keys.shape[-1]
attn_weigths = torch.softmax(atten_scores / d_k**0.5, dim=-1)

Compute_context_vector

In [90]:
context_vector = attn_weigths @ values
print("Context Vector:", context_vector.shape)

Context Vector: torch.Size([6, 2])


Implementation through object oriented programming

In [91]:
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.randn(d_in,d_out))
        self.W_key = nn.Parameter(torch.randn(d_in,d_out))
        self.W_value = nn.Parameter(torch.randn(d_in,d_out))

    def forward(self,x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        atten_scores = queries @ keys.T
        atten_weigths = torch.softmax(atten_scores / keys.shape[-1]**0.5, dim=-1)
        context_vector = atten_weigths @ values
        return context_vector

In [92]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]], grad_fn=<MmBackward0>)


Self_attention class implementation through torch.linear layer

For better weight initalization scheme

In [93]:
class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):   
        super().__init__()
        self.W_query = nn.Linear(d_in , d_out , bias = qkv_bias)
        self.W_key = nn.Linear(d_in , d_out , bias = qkv_bias)
        self.W_value = nn.Linear(d_in , d_out , bias = qkv_bias)
        

    def forward(self,x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        print("dim is :" , self.W_query(x).shape)

        atten_scores = queries @ keys.T
        atten_weigths = torch.softmax(atten_scores / keys.shape[-1]**0.5, dim=-1)
        context_vector = atten_weigths @ values
        return context_vector
    
    

In [94]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

dim is : torch.Size([6, 2])
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
